In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [4]:
for data in dataset:
    print(data)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [27]:
dataset = tf.data.Dataset.range(10)

In [28]:
for d in dataset:
    print(d)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [29]:
dataset = dataset.repeat(2).batch(6)
for d in dataset:
    print(d)

tf.Tensor([0 1 2 3 4 5], shape=(6,), dtype=int64)
tf.Tensor([6 7 8 9 0 1], shape=(6,), dtype=int64)
tf.Tensor([2 3 4 5 6 7], shape=(6,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


In [30]:
dataset =dataset.map(lambda x: x*2)
for d in dataset:
    print(d)

tf.Tensor([ 0  2  4  6  8 10], shape=(6,), dtype=int64)
tf.Tensor([12 14 16 18  0  2], shape=(6,), dtype=int64)
tf.Tensor([ 4  6  8 10 12 14], shape=(6,), dtype=int64)
tf.Tensor([16 18], shape=(2,), dtype=int64)


In [31]:
dataset = dataset.unbatch()
for d in dataset:
    print(d)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(10, shape=(), dtype=int64)
tf.Tensor(12, shape=(), dtype=int64)
tf.Tensor(14, shape=(), dtype=int64)
tf.Tensor(16, shape=(), dtype=int64)
tf.Tensor(18, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(10, shape=(), dtype=int64)
tf.Tensor(12, shape=(), dtype=int64)
tf.Tensor(14, shape=(), dtype=int64)
tf.Tensor(16, shape=(), dtype=int64)
tf.Tensor(18, shape=(), dtype=int64)


In [32]:
dataset = dataset.filter(lambda x: x<10)
for d in dataset:
    print(d)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)


In [33]:
for d in dataset.take(5):
    print(d)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)


In [38]:
dataset = tf.data.Dataset.range(10).repeat(2)
dataset = dataset.shuffle(buffer_size=6, seed=123).batch(3)
for d in dataset:
    print(d)

tf.Tensor([4 0 3], shape=(3,), dtype=int64)
tf.Tensor([8 2 9], shape=(3,), dtype=int64)
tf.Tensor([7 2 0], shape=(3,), dtype=int64)
tf.Tensor([3 5 6], shape=(3,), dtype=int64)
tf.Tensor([1 4 6], shape=(3,), dtype=int64)
tf.Tensor([9 7 5], shape=(3,), dtype=int64)
tf.Tensor([1 8], shape=(2,), dtype=int64)


In [39]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

In [44]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join('Datasets', 'housing')
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, 'my_{}_{:02d}.csv')
    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, 'wt', encoding='utf-8') as f:
            if header is not None:
                f.write(header)
                f.write('\n')
            for row_idx in row_indices:
                f.write(','.join([repr(col) for col in data[row_idx]]))
                f.write('\n')
    return filepaths

In [45]:
train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

In [46]:
train_filepaths

['Datasets\\housing\\my_train_00.csv',
 'Datasets\\housing\\my_train_01.csv',
 'Datasets\\housing\\my_train_02.csv',
 'Datasets\\housing\\my_train_03.csv',
 'Datasets\\housing\\my_train_04.csv',
 'Datasets\\housing\\my_train_05.csv',
 'Datasets\\housing\\my_train_06.csv',
 'Datasets\\housing\\my_train_07.csv',
 'Datasets\\housing\\my_train_08.csv',
 'Datasets\\housing\\my_train_09.csv',
 'Datasets\\housing\\my_train_10.csv',
 'Datasets\\housing\\my_train_11.csv',
 'Datasets\\housing\\my_train_12.csv',
 'Datasets\\housing\\my_train_13.csv',
 'Datasets\\housing\\my_train_14.csv',
 'Datasets\\housing\\my_train_15.csv',
 'Datasets\\housing\\my_train_16.csv',
 'Datasets\\housing\\my_train_17.csv',
 'Datasets\\housing\\my_train_18.csv',
 'Datasets\\housing\\my_train_19.csv']

In [47]:
import pandas as pd

pd.read_csv(train_filepaths[0]).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,3.5214,15.0,3.049945,1.106548,1447.0,1.605993,37.63,-122.43,1.442
1,5.3275,5.0,6.490060,0.991054,3464.0,3.443340,33.69,-117.39,1.687
2,3.1000,29.0,7.542373,1.591525,1328.0,2.250847,38.44,-122.98,1.621
3,7.1736,12.0,6.289003,0.997442,1054.0,2.695652,33.55,-117.70,2.621
4,2.0549,13.0,5.312457,1.085092,3297.0,2.244384,33.93,-116.93,0.956


In [49]:
with open(train_filepaths[0]) as f:
    for i in range(5):
        print(f.readline(), end='')

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442
5.3275,5.0,6.490059642147117,0.9910536779324056,3464.0,3.4433399602385686,33.69,-117.39,1.687
3.1,29.0,7.5423728813559325,1.5915254237288134,1328.0,2.2508474576271187,38.44,-122.98,1.621
7.1736,12.0,6.289002557544757,0.9974424552429667,1054.0,2.6956521739130435,33.55,-117.7,2.621


In [51]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=123)

for file in filepath_dataset:
    print(file)

tf.Tensor(b'Datasets\\housing\\my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_03.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_06.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_18.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_15.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_13.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_08.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_12.csv', shape=(), dtype=string)
tf.Tensor(b'Datasets\\housing\\my_train_09.csv', sh

In [52]:
n_readers = 5
dataset = filepath_dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
                                     cycle_length = n_readers)

In [53]:
for data in dataset.take(5):
    print(data)

tf.Tensor(b'4.2083,44.0,5.323204419889502,0.9171270718232044,846.0,2.3370165745856353,37.47,-122.2,2.782', shape=(), dtype=string)
tf.Tensor(b'3.8456,35.0,5.461346633416459,0.9576059850374065,1154.0,2.8778054862842892,37.96,-122.05,1.598', shape=(), dtype=string)
tf.Tensor(b'2.1856,41.0,3.7189873417721517,1.0658227848101265,803.0,2.0329113924050635,32.76,-117.12,1.205', shape=(), dtype=string)
tf.Tensor(b'5.9522,26.0,6.196521739130435,1.0069565217391305,1479.0,2.5721739130434784,34.5,-119.75,4.384', shape=(), dtype=string)
tf.Tensor(b'3.6875,44.0,4.524475524475524,0.993006993006993,457.0,3.195804195804196,34.04,-118.15,1.625', shape=(), dtype=string)


In [58]:
c=0
for data in dataset:
    c+=1

print(c)

11610
